In [1]:
"""
PyTorch Giff Dataset Class for loading gifs using Pytorch DataLoader.
This Dataset resizes all frames to a specific size.

Facundo Calcagno
"""
from __future__ import print_function, division
import cv2
import os
import torch
import numpy as np
import pickle
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image,ImageFilter , ImageOps, ImageDraw

In [7]:
class gifDataSet(Dataset):
    """Dataset Class for loading Gif into a 3D Tensor"""
    def __init__(self,gifList,rootDir, channels, timeDepth, xSize, ySize, startFrame,endFrame,transform=None):
        """
        Args:
        clipsList (string): Path to the clipsList file with labels.
        rootDir (string): Directory with all the gifs.
        transform (callable, optional): Optional transform to be applied
            on a sample.
        channels: Number of channels of frames
        timeDepth: Number of frames to be loaded in a sample
        xSize, ySize: Dimensions of the frames
        mean: Mean valuse of the training set videos over each channel
        """
        self.gifList = gifList
        self.rootDir = rootDir
        self.channels = channels
        self.timeDepth = timeDepth
        self.xSize = xSize
        self.ySize = ySize
        #self.mean = mean
        self.transform = transform
        self.startFrame=startFrame
        self.endFrame=endFrame
    
    def __len__(self):
        return len(self.gifList)
    
    def crop6(self,im):
        number_of_cols=3
        W=im.width
        H=im.height
        w=(W-16)/3
        h=(H-24)/2
        images=[]
        w1=0
        w2=w
        for i in range(number_of_cols):
            im1=im.crop((w1, 0, w2, h))
            images.append(im1)
            im1=im.crop((w1, h+8, w2, 2*h+8))
            images.append(im1)
            w1=w2+8
            w2=w2+w+8
        return images
    
    def readGif(self, gifFile):
        # Open the gif file, crop it and return the frames in a tensor
        #cap= cv2.VideoCapture(gifFile)
        image_gif=Image.open(self.rootDir+ gifFile, mode='r')
        frames = torch.FloatTensor(self.timeDepth, self.channels, self.xSize, self.ySize)
        nframes = 0
        nframesin=0
        while image_gif:
            if self.startFrame<nframes<self.endFrame:
                six_images=self.crop6(image_gif)
                pil_image = six_images[4]
                print(np.array(pil_image).shape)
                imResize=pil_image.resize((self.xSize, self.ySize),Image.ANTIALIAS)
                print(np.array(imResize).shape)
                frame = torch.from_numpy(np.array(imResize))
                #frame = torch.from_numpy(np.array(frame))
                #frame = frame.permute(2, 0, 1)
                print(frame.size())
                frames[nframesin, :, :, :] = frame
                nframesin+=1
            nframes += 1
            try:
                image_gif.seek( nframes )
            except EOFError:
                break;
            
        image_gif.close()
        return frames

In [8]:
rootDir="gifs/"
import glob
mypath=rootDir+'Mediophyceae_Hemiaulus'
a=glob.glob(rootDir+'*.gif')
#with open('gifs.pickle', 'wb') as fp:
#    pickle.dump(a, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
gifListFile='gifs.pickle'
rootDir="gifs/"
channels=1
timeDepth=3
xSize=100
ySize=100
startFrame=1
endFrame=5

In [10]:
gifDataset= gifDataSet(a,rootDir,channels,timeDepth,xSize,ySize,startFrame,endFrame)

In [11]:
tensor=gifDataset.readGif("10753468_S127--D0--R27--G100010824--A131203--L02236--animation.gif")

(180, 193)
(100, 100)
torch.Size([100, 100])
(180, 193)
(100, 100)
torch.Size([100, 100])
(180, 193)
(100, 100)
torch.Size([100, 100])


In [15]:
tensor[2]

tensor([[[ 194.,  195.,  194.,  ...,  195.,  194.,  194.],
         [ 194.,  194.,  195.,  ...,  194.,  195.,  195.],
         [ 194.,  195.,  194.,  ...,  223.,  194.,  223.],
         ...,
         [ 195.,  194.,  195.,  ...,  195.,  194.,  223.],
         [ 195.,  223.,  194.,  ...,  195.,  195.,  195.],
         [ 195.,  194.,  195.,  ...,  194.,  223.,  223.]]])